### This notebook demonstrates the use of an odds-equalizing post-processing algorithm for bias mitigiation

In [44]:
%load_ext autoreload
%autoreload 2
%aimport logistic_regression_model
%aimport load_dataset
%aimport fairness_metrics

import sys
import os
import numpy as np
import torch
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
import calibrated_eq_odds_postprocessing
from IPython.display import Markdown, display

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
# INPUT PARAMS
LABEL_COL, PROTECT_COLS, MODE, START_EPOCH, NUM_EPOCH, ID, NUM_TRIALS, NUM_PROXIES, FILE_PATH, VERBOSE, \
LR_RATE, UPDATE, WEIGHTS_INIT, UPDATE_LR, BATCH_SIZE, BALANCE = "income", ["gender"], 0, 0, 40, -1, 1, 0, \
                                                                "../Datasets/adult_dataset/processed_adult.csv", 1, \
                                                                0.001, "cluster", 0, 10, 1000, 0

In [46]:
privileged_groups = [{'sex': 0}]
unprivileged_groups = [{'sex': 1}]
favorable_label, unfavorable_label = 1, 0


balanced = {"train_label_only": True, "test_label_only": False, "downsample": True} if BALANCE else None


df = load_dataset.get_data(FILE_PATH)
df = load_dataset.minmax_scale(df)

train_df, test_df = load_dataset.split_train_test(df, train=0.75)
val_df, test_df = load_dataset.split_train_test(test_df, train=0.5)

if balanced is not None:
    train_df = load_dataset.balance_df(df, LABEL_COL, PROTECT_COLS, label_only=balanced["train_label_only"],
                            downsample=balanced["downsample"])
    val_df = load_dataset.balance_df(df, LABEL_COL, PROTECT_COLS, label_only=balanced["train_label_only"],
                            downsample=balanced["downsample"])
    test_df = load_dataset.balance_df(df, LABEL_COL, PROTECT_COLS, label_only=balanced["test_label_only"],
                            downsample=balanced["downsample"])

# Splitting dataset into train, test features
print("Statistics")
load_dataset.statistics(train_df, LABEL_COL, PROTECT_COLS, verbose=1)
load_dataset.statistics(val_df, LABEL_COL, PROTECT_COLS, verbose=1)
load_dataset.statistics(test_df, LABEL_COL, PROTECT_COLS, verbose=1)

dataset_orig_train = load_dataset.Dataset(train_df, LABEL_COL, PROTECT_COLS)
dataset_orig_valid = load_dataset.Dataset(val_df, LABEL_COL, PROTECT_COLS)
dataset_orig_test = load_dataset.Dataset(test_df, LABEL_COL, PROTECT_COLS)


# Metric used (should be one of allowed_metrics)
metric_name = "Average odds difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
np.random.seed(1)

print("---------- MAPPING ----------")
print("Train: ", dataset_orig_train.mapping)
print("Val: ", dataset_orig_valid.mapping)
print("Test: ", dataset_orig_test.mapping)
print("-----------------------------")


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


Statistics
{0.0: gender
0.0     (9798, 0.384)
1.0    (15705, 0.616)
dtype: object, 1.0: gender
0.0    (1250, 0.149)
1.0    (7163, 0.851)
dtype: object}
label: 0.0: 25503 samples (75.19%)
label: 1.0: 8413 samples (24.81%)
{1.0: gender
0.0     (202, 0.139)
1.0    (1248, 0.861)
dtype: object, 0.0: gender
0.0    (1597, 0.38)
1.0    (2606, 0.62)
dtype: object}
label: 1.0: 1450 samples (25.65%)
label: 0.0: 4203 samples (74.35%)
{0.0: gender
0.0    (1631, 0.379)
1.0    (2677, 0.621)
dtype: object, 1.0: gender
0.0     (217, 0.161)
1.0    (1128, 0.839)
dtype: object}
label: 0.0: 4308 samples (76.21%)
label: 1.0: 1345 samples (23.79%)
---------- MAPPING ----------
Train:  {(0.0,): 0, (1.0,): 1}
Val:  {(1.0,): 0, (0.0,): 1}
Test:  {(0.0,): 0, (1.0,): 1}
-----------------------------


#### Train classifier (logistic regression on original training data)¶


In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

# Placeholder for predicted and transformed datasets
dataset_orig_train_pred = dataset_orig_train.copy()
dataset_orig_valid_pred = dataset_orig_valid.copy()
dataset_orig_test_pred = dataset_orig_test.copy()

dataset_new_valid_pred = dataset_orig_valid.copy()
dataset_new_test_pred = dataset_orig_test.copy()

# Logistic regression classifier and predictions for training data
X_train = dataset_orig_train.features
y_train = dataset_orig_train.label.ravel()
lmod = LogisticRegression()
lmod.fit(X_train, y_train)

fav_idx = np.where(lmod.classes_ == favorable_label)[0][0]
y_train_pred_prob = lmod.predict_proba(X_train)[:,fav_idx]

# Prediction probs for validation and testing data
X_valid = dataset_orig_valid.features
y_valid_pred_prob = lmod.predict_proba(X_valid)[:,fav_idx]

X_test = dataset_orig_test.features
y_test_pred_prob = lmod.predict_proba(X_test)[:,fav_idx]

class_thresh = 0.5
train_scores = y_train_pred_prob.reshape(-1,1)
val_scores = y_valid_pred_prob.reshape(-1,1)
test_scores = y_test_pred_prob.reshape(-1,1)

y_train_pred = np.zeros_like(dataset_orig_train_pred.label)
y_train_pred[y_train_pred_prob >= class_thresh] = favorable_label
y_train_pred[~(y_train_pred_prob >= class_thresh)] = unfavorable_label
dataset_orig_train_pred.label = y_train_pred

y_valid_pred = np.zeros_like(dataset_orig_valid_pred.label)
y_valid_pred[y_valid_pred_prob >= class_thresh] = favorable_label
y_valid_pred[~(y_valid_pred_prob >= class_thresh)] = unfavorable_label
dataset_orig_valid_pred.label = y_valid_pred
    
y_test_pred = np.zeros_like(dataset_orig_test_pred.label)
y_test_pred[y_test_pred_prob >= class_thresh] = favorable_label
y_test_pred[~(y_test_pred_prob >= class_thresh)] = unfavorable_label
dataset_orig_test_pred.label = y_test_pred

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [48]:

display(Markdown("#### Original-Predicted training dataset"))

acc = np.sum(dataset_orig_train.label.reshape(-1) == dataset_orig_train_pred.label.reshape(-1))/len(dataset_orig_train.label)
odds = fairness_metrics.equalizing_odds(dataset_orig_train_pred.label, dataset_orig_train.label,
                                                       dataset_orig_train_pred.protect)
diffs = [max(odd) - min(odd) for odd in odds]
print(f"Accuracy: {acc*100}%")
print(f"Equalizing Odds: {odds}")
print(f"Weighted average odds difference", np.average(diffs, weights=[np.sum(dataset_orig_train.label == unfavorable_label), np.sum(dataset_orig_train.label == favorable_label)]))


display(Markdown("#### Original-Predicted validation dataset"))

acc = np.sum(dataset_orig_valid.label.reshape(-1) == dataset_orig_valid_pred.label.reshape(-1))/len(dataset_orig_valid.label)
odds = fairness_metrics.equalizing_odds(dataset_orig_valid_pred.label, dataset_orig_valid.label,
                                                       dataset_orig_valid.protect)
diffs = [max(odd) - min(odd) for odd in odds]
print(f"Accuracy: {acc*100}%")
print(f"Equalizing Odds: {odds}")
print(f"Weighted average odds difference", np.average(diffs, weights=[np.sum(dataset_orig_valid.label == unfavorable_label), np.sum(dataset_orig_valid.label == favorable_label)]))
display(Markdown("#### Original-Predicted testing dataset"))

acc = np.sum(dataset_orig_test_pred.label.reshape(-1) == dataset_orig_test.label.reshape(-1))/len(dataset_orig_test.label)
odds = fairness_metrics.equalizing_odds(dataset_orig_test_pred.label, dataset_orig_test.label,
                                                       dataset_orig_test_pred.protect)
diffs = [max(odd) - min(odd) for odd in odds]
print(f"Accuracy: {acc*100}%")
print(f"Equalizing Odds: {odds}")
print(f"Weighted average odds difference", np.average(diffs, weights=[np.sum(dataset_orig_test.label == unfavorable_label), np.sum(dataset_orig_test.label == favorable_label)]))

#### Original-Predicted training dataset

Accuracy: 84.9451586271966%
Equalizing Odds: [[0.979, 0.898], [0.495, 0.626]]
Weighted average odds difference 0.09340270079018749


#### Original-Predicted validation dataset

Accuracy: 84.73376967981604%
Equalizing Odds: [[0.9, 0.977], [0.631, 0.475]]
Weighted average odds difference 0.09726357686184325


#### Original-Predicted testing dataset

Accuracy: 85.74208384928357%
Equalizing Odds: [[0.984, 0.905], [0.558, 0.618]]
Weighted average odds difference 0.07447939147355383


### Perform odds equalizing post processing on scores

In [54]:
# Odds equalizing post-processing algorithm
from tqdm import tqdm

cost_constraint = "weighted"
randseed = 12345679

# Learn parameters to equalize odds and apply to create a new dataset
cpp = calibrated_eq_odds_postprocessing.CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                     unprivileged_groups = unprivileged_groups,
                                     cost_constraint=cost_constraint,
                                     seed=randseed)
cpp = cpp.fit(dataset_orig_valid, val_scores)

### Transform validation and test data using the post processing algorithm

In [56]:
dataset_transf_valid_pred, new_val_scores = cpp.predict(dataset_orig_valid_pred, val_scores)
dataset_transf_test_pred, new_test_scores = cpp.predict(dataset_orig_test_pred, test_scores)

In [59]:
display(Markdown("#### Original-Predicted validation dataset"))

acc = np.sum(dataset_orig_valid.label.reshape(-1) == dataset_transf_valid_pred.label.reshape(-1))/len(dataset_orig_valid.label)
odds = fairness_metrics.equalizing_odds(dataset_transf_valid_pred.label, dataset_orig_valid.label,
                                                       dataset_orig_valid.protect)
diffs = [max(odd) - min(odd) for odd in odds]
print(f"Accuracy: {acc*100}%")
print(f"Equalizing Odds: {odds}")
print(f"Weighted average odds difference", np.average(diffs, weights=[np.sum(dataset_orig_valid.label == unfavorable_label), np.sum(dataset_orig_valid.label == favorable_label)]))
display(Markdown("#### Original-Predicted testing dataset"))

acc = np.sum(dataset_transf_test_pred.label.reshape(-1) == dataset_orig_test.label.reshape(-1))/len(dataset_orig_test.label)
odds = fairness_metrics.equalizing_odds(dataset_transf_test_pred.label, dataset_orig_test.label,
                                                       dataset_transf_test_pred.protect)
diffs = [max(odd) - min(odd) for odd in odds]
print(f"Accuracy: {acc*100}%")
print(f"Equalizing Odds: {odds}")
print(f"Weighted average odds difference", np.average(diffs, weights=[np.sum(dataset_orig_test.label == unfavorable_label), np.sum(dataset_orig_test.label == favorable_label)]))

#### Original-Predicted validation dataset

Accuracy: 83.70776578807713%
Equalizing Odds: [[0.9, 0.998], [0.631, 0.025]]
Weighted average odds difference 0.22830249425084023


#### Original-Predicted testing dataset

Accuracy: 78.6308154961967%
Equalizing Odds: [[0.984, 0.993], [0.558, 0.055]]
Weighted average odds difference 0.1265358216875995
